## 07 Optimization

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

### Hyperparameters

In [2]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

### Loss Function

In [3]:
loss_fn = nn.CrossEntropyLoss()

### Learning Rate

In [5]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

### Train

In [7]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred, y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        if batch % 100 == 0:
            print(f"loss: {loss:>7f}  [{(batch+1)*len(X)}/{size}]")

In [8]:
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    
    test_loss, correct = 0, 0
    with torch.no_grad():
        for batch, (X, y) in enumerate(dataloader):
            pred = model(X)
            test_loss += loss_fn(pred, y)
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [9]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.307507  [64/60000]
loss: 2.294358  [6464/60000]
loss: 2.278862  [12864/60000]
loss: 2.268537  [19264/60000]
loss: 2.261932  [25664/60000]
loss: 2.238371  [32064/60000]
loss: 2.238500  [38464/60000]
loss: 2.214792  [44864/60000]
loss: 2.205155  [51264/60000]
loss: 2.184696  [57664/60000]
Test Error: 
 Accuracy: 46.4%, Avg loss: 2.175767 

Epoch 2
-------------------------------
loss: 2.185969  [64/60000]
loss: 2.174236  [6464/60000]
loss: 2.127946  [12864/60000]
loss: 2.135188  [19264/60000]
loss: 2.094118  [25664/60000]
loss: 2.046141  [32064/60000]
loss: 2.062398  [38464/60000]
loss: 1.999095  [44864/60000]
loss: 1.999969  [51264/60000]
loss: 1.933496  [57664/60000]
Test Error: 
 Accuracy: 55.2%, Avg loss: 1.932105 

Epoch 3
-------------------------------
loss: 1.963081  [64/60000]
loss: 1.932585  [6464/60000]
loss: 1.832553  [12864/60000]
loss: 1.859150  [19264/60000]
loss: 1.752367  [25664/60000]
loss: 1.710954  [32064/60000]
loss: 1.